# No EDA here, thats on the KNN with simple features.

# This is just testing some feature engineering which works with the same changes as "KNN with simple feature" i am just changing step 5 and 6 which is the Multivariate Analysis and the conclusion

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import numpy as np
import scipy.stats as stats
import category_encoders as ce


# Load the CSV file
file_path = 'AB_NYC_2019.csv'
Airbnb_data_df = pd.read_csv(file_path)

# View the dataframe, show all columns and give all numeric columns as floats. So its more readable
print("Data Overview:")
pd.set_option('display.max_columns', None)

display(Airbnb_data_df)

Data Overview:


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [16]:
# Removing the above data - As they are not and have most likely never been on the market. Explained in other EDA
Airbnb_data_df = Airbnb_data_df[
    ~((Airbnb_data_df['availability_365'] == 0) & (Airbnb_data_df['number_of_reviews'] == 0))
].reset_index(drop=True)
Airbnb_data_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44045,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
44046,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
44047,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
44048,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [17]:
# Fort Wadsworth Removing as it has no reviews and only 1 listing. making it an outlier.
Airbnb_data_df = Airbnb_data_df[Airbnb_data_df['neighbourhood'] != 'Fort Wadsworth'].reset_index(drop=True)
print('Final Dataframe')
Airbnb_data_df

Final Dataframe


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44044,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
44045,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
44046,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
44047,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


In [24]:
# ▌Airbnb Neighbourhood Recommender – JupyterLab version (final with debug prints)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import traceback

# 1️⃣ Load & prepare
df = Airbnb_data_df.copy()
df["reviews_per_month"] = df["reviews_per_month"].fillna(0)
df["busyness_score"] = (
    (365 - df["availability_365"])           # fewer free days ⇒ busier
    + df["number_of_reviews"]
    + df["reviews_per_month"] * 12           # month → annualised count
)
# 2️⃣ Aggregate
agg_base = df.groupby(["neighbourhood_group", "neighbourhood"]).agg(
    avg_price=("price", "mean"),
    avg_min_nights=("minimum_nights", "mean"),
    mean_busyness=("busyness_score", "mean")
).reset_index()

room_type_mode = df.groupby(["neighbourhood_group", "neighbourhood"])["room_type"] \
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0]).reset_index()

agg = pd.merge(agg_base, room_type_mode, on=["neighbourhood_group", "neighbourhood"])

# Flag busy/less_busy
q75 = agg["mean_busyness"].quantile(0.75)
q25 = agg["mean_busyness"].quantile(0.25)
agg["busy_flag"] = np.where(agg["mean_busyness"] >= q75, "busy", "less_busy")

# 🔎 Print range before interface
print(f"📊 Busyness score range:")
print(f"- Min: {agg['mean_busyness'].min():.1f}")
print(f"- 25th percentile (less busy <): {q25:.1f}")
print(f"- 75th percentile (busy >=): {q75:.1f}")
print(f"- Max: {agg['mean_busyness'].max():.1f}")

# 3️⃣ Similarity matrix
nbr_features = ["avg_price", "avg_min_nights", "mean_busyness"]
nbr_matrix = StandardScaler().fit_transform(agg[nbr_features])
nn = NearestNeighbors(n_neighbors=min(10, len(agg) - 1)).fit(nbr_matrix)
coords_2d = PCA(n_components=2).fit_transform(nbr_matrix)

# 4️⃣ Widgets
group_dd = widgets.Dropdown(options=sorted(agg["neighbourhood_group"].unique()),
                            description="Group:", layout=widgets.Layout(width="40%"))

neigh_dd = widgets.Dropdown(description="Neighbourhood:", disabled=True,
                            layout=widgets.Layout(width="60%"))

output_box = widgets.Output()
run_button = widgets.Button(description="🔍 Find Similar Neighbourhoods", button_style='info',
                            layout=widgets.Layout(width="50%"))

def update_neigh_dd(*_):
    selected = group_dd.value
    options = sorted(agg.loc[agg["neighbourhood_group"] == selected, "neighbourhood"])
    neigh_dd.options = options
    neigh_dd.disabled = False if options else True
    if options:
        neigh_dd.value = options[0]

group_dd.observe(update_neigh_dd, names="value")
group_dd.value = group_dd.options[0]  # Trigger fill
update_neigh_dd()

# 5️⃣ Recommender logic
def recommend_and_plot(button_event):
    with output_box:
        output_box.clear_output()
        try:
            group = group_dd.value
            neigh = neigh_dd.value

            match = agg[(agg.neighbourhood_group == group) & (agg.neighbourhood == neigh)]
            if match.empty:
                display(Markdown("❌ **Neighbourhood not found.**"))
                return

            idx = match.index[0]
            dists, idxs = nn.kneighbors(nbr_matrix[idx].reshape(1, -1))
            similar_df = agg.loc[idxs.flatten()].copy()
            similar_df["distance"] = dists.flatten()
            similar_df = similar_df[similar_df["neighbourhood"] != neigh].head(3)

            sel = match.iloc[0]
            display(Markdown(f"## 🔎 **{neigh}** in **{group}**"))
            display(Markdown(
                f"- 💰 **${sel.avg_price:.0f}** / night  \n"
                f"- 🛎 **{sel.avg_min_nights:.0f}** min nights  \n"
                f"- 🏷 **Room Type:** {sel.room_type}  \n"
                f"- 🔊 **Busyness:** {sel.mean_busyness:.1f} ({sel.busy_flag})"
            ))

            display(Markdown("### 📊 Top 3 Most Similar Neighbourhoods"))
            for _, r in similar_df.iterrows():
                flag = "✅" if r.busy_flag == "less_busy" else "⚠️"
                display(Markdown(
                    f"{flag} **{r.neighbourhood}** — Distance: {r.distance:.3f}  \n"
                    f"💰 ${r.avg_price:.0f}, 🛎 {r.avg_min_nights:.0f} nights, "
                    f"{r.room_type}, 🔊 {r.mean_busyness:.1f}"
                ))

            # 🔍 Most similar less busy (even if not top 3)
            less_busy_sorted = agg.loc[idxs.flatten()].copy()
            less_busy_sorted["distance"] = dists.flatten()
            less_busy = less_busy_sorted[
                (less_busy_sorted["neighbourhood"] != neigh) &
                (less_busy_sorted["busy_flag"] == "less_busy")
            ].sort_values("distance")

            if not less_busy.empty:
                best_less_busy = less_busy.iloc[0]
                display(Markdown("### ✅ Closest *Less Busy* Option"))
                display(Markdown(
                    f"**{best_less_busy.neighbourhood}** in **{best_less_busy.neighbourhood_group}**  \n"
                    f"- 🔊 {best_less_busy.mean_busyness:.1f} (less busy)  \n"
                    f"- 💰 ${best_less_busy.avg_price:.0f}, 🛎 {best_less_busy.avg_min_nights:.0f} nights, "
                    f"{best_less_busy.room_type}"
                ))

            # Plot
            plt.figure(figsize=(10, 6))
            sns.scatterplot(x=coords_2d[:, 0], y=coords_2d[:, 1],
                            hue=agg.busy_flag, style=agg.neighbourhood_group,
                            palette={"busy": "red", "less_busy": "green"}, alpha=0.5)
            plt.scatter(*coords_2d[idx], s=250, edgecolor='black',
                        facecolor='yellow', label="Selected")

            for _, r in similar_df.iterrows():
                i = r.name
                plt.scatter(*coords_2d[i], s=180, edgecolor='black',
                            facecolor='blue', label=f"{r['neighbourhood']}")

            plt.title("Neighbourhood Similarity (PCA Projection)")
            plt.xlabel("Component 1")
            plt.ylabel("Component 2")
            plt.grid(True)
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.tight_layout()
            plt.show()

        except Exception as e:
            traceback.print_exc()

run_button.on_click(recommend_and_plot)

# 6️⃣ Interface
display(Markdown("## Select a neighbourhood"))
display(widgets.HBox([group_dd, neigh_dd]))
display(run_button)
display(output_box)


📊 Busyness score range:
- Min: 20.9
- 25th percentile (less busy <): 213.4
- 75th percentile (busy >=): 288.3
- Max: 373.6


## Select a neighbourhood

Button(button_style='info', description='🔍 Find Similar Neighbourhoods', layout=Layout(width='50%'), style=But…

Output()